In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeClassifierCV
from r_modified_functions import generate_kernels, transform_and_select_features
from sktime.datasets import load_UCR_UEA_dataset
import time

In [8]:
dataset_names = ["Adiac",
    "ArrowHead",
    "Beef",
    "BeetleFly",
    "BirdChicken",
    "Car",
    "CBF",
    "ChlorineConcentration",
    "CinCECGTorso",
    "Coffee",
    "Computers",
    "CricketX",
    "CricketY",
    "CricketZ",
    "DistalPhalanxOutlineCorrect",
    "DistalPhalanxOutlineAgeGroup",
    "DistalPhalanxTW",
    "Earthquakes",
    "ECG200",
    "ECGFiveDays",
    "ElectricDevices",
    "FaceAll",
    "FacesUCR",
    "Fish",
    "FordA",
    "FordB",
    "GunPoint",
    "Ham",
    "HandOutlines",
    "Haptics",
    "Herring",
    "InlineSkate",
    "InsectWingbeatSound",
    "ItalyPowerDemand",
    "LargeKitchenAppliances",
    "Lightning2",
    "Lightning7",
    "Meat",
    "MedicalImages",
    "MiddlePhalanxOutlineCorrect",
    "MiddlePhalanxOutlineAgeGroup",
    "MiddlePhalanxTW",
    "MoteStrain",
    "NonInvasiveFetalECGThorax1",
    "NonInvasiveFetalECGThorax2",
    "OliveOil",
    "OSULeaf",
    "PhalangesOutlinesCorrect",
    "Plane",
    "ProximalPhalanxOutlineCorrect",
    "ProximalPhalanxOutlineAgeGroup",
    "ProximalPhalanxTW",
    "RefrigerationDevices",
    "ScreenType",
    "ShapeletSim",
    "ShapesAll",
    "SmallKitchenAppliances",
    "SonyAIBORobotSurface1",
    "SonyAIBORobotSurface2",
    "StarLightCurves",
    "Strawberry",
    "SwedishLeaf",
    "SyntheticControl",
    "ToeSegmentation1",
    "ToeSegmentation2",
    "Trace",
    "TwoLeadECG",
    "TwoPatterns",
    "UWaveGestureLibraryX",
    "UWaveGestureLibraryY",
    "UWaveGestureLibraryZ",
    "UWaveGestureLibraryAll",
    "Wafer",
    "Wine",
    "Worms",
    "WormsTwoClass",
    "Yoga"]

In [ ]:
results = []
for dataset_name in dataset_names:
    print(f"Processing dataset: {dataset_name}")
    X_train, y_train = load_UCR_UEA_dataset(dataset_name, split="train", return_X_y=True)
    X_test, y_test = load_UCR_UEA_dataset(dataset_name, split="test", return_X_y=True)

    # Convert DataFrame to numpy array if necessary
    if isinstance(X_train, pd.DataFrame):
        X_train = np.stack(X_train.iloc[:, 0].apply(lambda x: x.to_numpy() if isinstance(x, pd.Series) else x))
    if isinstance(X_test, pd.DataFrame):
        X_test = np.stack(X_test.iloc[:, 0].apply(lambda x: x.to_numpy() if isinstance(x, pd.Series) else x))

    avg_series_length = np.mean([len(x) for x in X_train])

    # Generate kernels
    kernels = generate_kernels(X_train.shape[1], 10000, int(avg_series_length))

    # Transform and select features for training set
    X_train_transformed, selector, best_num_features, scaler = transform_and_select_features(X_train, kernels, y_train, is_train=True)

    # Transform and select features for test set using the same selector
    X_test_transformed = transform_and_select_features(X_test, kernels, selector=selector, scaler=scaler,
                                                       is_train=False)

    # Classifier
    classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))

    # Train classifier
    start_time = time.time()
    classifier.fit(X_train_transformed, y_train)
    training_time = time.time() - start_time

    # Test classifier
    start_time = time.time()
    predictions = classifier.predict(X_test_transformed)
    test_time = time.time() - start_time
    accuracy = np.mean(predictions == y_test)

    results.append({
        "Dataset": dataset_name,
        "Accuracy": accuracy,
        "Training Transformation Time": time.time() - start_time,
        "Training Time": training_time,
        "Test Time": test_time,
    })

    print(f"Dataset: {dataset_name}, Accuracy: {accuracy}")
    print(f"Training Transformation Time: {time.time() - start_time}s")
    print(f"Training Time: {training_time}s, Test Time: {test_time}s")
    print("=" * 50)

Processing dataset: Adiac
Dataset: Adiac, Accuracy: 0.7570332480818415
Training Transformation Time: 0.005011081695556641s
Training Time: 0.1532435417175293s, Test Time: 0.004004955291748047s
Processing dataset: ArrowHead
Dataset: ArrowHead, Accuracy: 0.7657142857142857
Training Transformation Time: 0.0010013580322265625s
Training Time: 0.01553487777709961s, Test Time: 0.0010013580322265625s
Processing dataset: Beef
Dataset: Beef, Accuracy: 0.7
Training Transformation Time: 0.0019996166229248047s
Training Time: 0.013011932373046875s, Test Time: 0.0010004043579101562s
Processing dataset: BeetleFly
Dataset: BeetleFly, Accuracy: 0.85
Training Transformation Time: 0.0009996891021728516s
Training Time: 0.007002592086791992s, Test Time: 0.0009996891021728516s
Processing dataset: BirdChicken
Dataset: BirdChicken, Accuracy: 0.95
Training Transformation Time: 0.0010001659393310547s
Training Time: 0.005995273590087891s, Test Time: 0.0s
Processing dataset: Car
Dataset: Car, Accuracy: 0.9
Training